In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utility_funcs import *
import xgboost as xgb

In [3]:
demanda_df = pd.read_csv(".\Datos\demanda.csv")

In [4]:
demanda_prod_df = demanda_df[demanda_df['id_producto']==1]
demanda_prod_df.drop(columns=['id_producto'], inplace=True)
demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')

# Primero, asegurar que todas las fechas estan
# Los datos de demanda iniciales no tienen enero 2020
fecha_inicial = pd.to_datetime('2020-01-01')
fechas_df = pd.DataFrame({'date': pd.date_range(start=fecha_inicial, end=demanda_prod_df['date'].max())})
complete_demanda_prod_df = fechas_df.merge(demanda_prod_df
                                           ,how='left'
                                           ,left_on='date'
                                           ,right_on='date'
                                           )

C:\Users\juanc\AppData\Local\Temp\ipykernel_31828\2870044259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df.drop(columns=['id_producto'], inplace=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_31828\2870044259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')


In [5]:
# Preprocesamiento de la serie de tiempo
complete_demanda_prod_df['demanda'].ffill(inplace=True)
complete_demanda_prod_df['demanda'].bfill(inplace=True)


# Eliminacion de 29 de febrero.
complete_demanda_prod_df = complete_demanda_prod_df.sort_values(by='date', ascending=True)

C:\Users\juanc\AppData\Local\Temp\ipykernel_31828\1117770948.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  complete_demanda_prod_df['demanda'].ffill(inplace=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_31828\1117770948.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [6]:
# Generacion de variables exogenas
exog_vars_dict = generate_exogenous(complete_demanda_prod_df['date'].min(), complete_demanda_prod_df.shape[0])
exog_vars_df = pd.DataFrame(exog_vars_dict)
exog_vars_df['date'] = complete_demanda_prod_df['date']

In [8]:
# Unioin de dataset general con exogenas
full_data_df = complete_demanda_prod_df.merge(exog_vars_df ,how='left' ,left_on='date' ,right_on='date', suffixes=('_endog', '_exog'))

In [11]:
# Extraer mes y anio (y dia)
full_data_df['anio'] = full_data_df['date'].dt.year
full_data_df['mes'] = full_data_df['date'].dt.month
full_data_df['dia'] = full_data_df['date'].dt.day
# Agrupar por mes, anio y luego calcular el Q1, Q3 y el IQR
grouped_df = full_data_df.groupby(['anio','mes'])['demanda'].agg(
        q1=lambda x: x.quantile(0.25),
        q3=lambda x: x.quantile(0.75)
).reset_index()

grouped_df['iqr'] = grouped_df['q3'] - grouped_df['q1']
grouped_df['lower_bound'] = grouped_df['q1'] - 1.5 * grouped_df['iqr']
grouped_df['upper_bound'] = grouped_df['q3'] + 1.5 * grouped_df['iqr']

# Hacer merge para unir los límites de cada mes con los datos originales
merged_df = pd.merge(full_data_df, grouped_df, on=['anio', 'mes'], how='left')

# Marcar outliers
merged_df['is_outlier_low'] = merged_df['demanda'] < merged_df['lower_bound']
merged_df['is_outlier_high'] = merged_df['demanda'] > merged_df['upper_bound']


In [14]:
merged_df.to_csv('ventas.csv',index=False)